In [1]:
pip install konlpy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 27.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Kkma, Komoran, Okt, Hannanum  #Mecab은 윈도우에서 작동 불가능

from konlpy import tag
from konlpy.corpus import kolaw
# from konlpy.utils import csvwrite, pprint

from nltk.tokenize import sent_tokenize

In [4]:
Lyrics = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/Lyrics_final.csv')

In [5]:
# 행복, 희망 => 1(긍정), 슬픔, 분노 => 0(부정) 으로 이진분류
def rating_to_label(label):
    if label == 0 or label == 3:
        return 1
    else:
        return 0

Lyrics['y'] = Lyrics['label'].apply(lambda x: rating_to_label(x))
Lyrics_pos = Lyrics[Lyrics['y'] == 1]
Lyrics_neg = Lyrics[Lyrics['y'] == 0]

In [6]:
kosac = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/odatt/polarity.csv")

### 태깅

In [7]:
okt = Okt() 
kkma = Kkma() #kosac에 알맞는 데이터 값
komoran = Komoran()
hannanum = Hannanum()

kkma 이용

In [8]:
data_pos = Lyrics_pos['Lyrics'].to_string()
data_neg = Lyrics_neg['Lyrics'].to_string()

kkma_pos = kkma.pos(data_pos)
kkma_neg = kkma.pos(data_neg)


# 형태소 필터링 

# kkma_text = ['NNG', 'NNP', 'NNB','VV','VA','MAG','IC','NR']
# clean_words_pos = []
# for i in kkma_pos:
#   for j in kkma_text:
#     if j in i:
#       clean_words_pos.append(i)

# clean_words_neg = []
# for i in kkma_neg:
#   for j in kkma_text:
#     if j in i:
#       clean_words_neg.append(i)


In [9]:
# 단어+형태소 합치기 

kkma_pos_list = []
kkma_neg_list = []

for i in range(len(kkma_pos)):
  pos_text = kkma_pos[i][0]+'/'+kkma_pos[i][1]
  kkma_pos_list.append(pos_text)
  i += 1

for i in range(len(kkma_neg)):
  neg_text = kkma_neg[i][0]+'/'+kkma_neg[i][1]
  kkma_neg_list.append(neg_text)
  i += 1


In [113]:
# 형태소분석이 된 긍정가사를 가사별로 리스트화하여 dataframe으로 정리
text_pos = ''.join(str(i) for i in kkma_pos_list)
text_neg = ' '.join(str(i) for i in kkma_neg_list)

text_pos_list = []
numList_pos = []

pattern = "[0-9]"

numList = re.findall(r'\d+', text_pos)

for i in numList:
  if(text_pos.find(f'{i}') > -1):
    num_pos = text_pos.find(f'{i}')
    numList_pos.append(num_pos)

for i in range(len(numList_pos)):
  if(i == len(numList_pos)-1):
    text_2 = text_pos[numList_pos[i]:len(text_pos)]
  else:
    text_2 = text_pos[numList_pos[i]:numList_pos[i+1]]
  text_pos_list.append(text_2)

pos = pd.DataFrame(text_pos_list)

# 형태소분석이 된 부정가사를 가사별로 리스트화하여 dataframe으로 정리
text_neg_list = []
numList_neg = []

pattern = "[0-9]"

numList = re.findall(r'\d+', text_neg)

for i in numList:
  if(text_neg.find(f'{i}') > -1):
    num_neg = text_neg.find(f'{i}')
    numList_neg.append(num_neg)

for i in range(len(numList_neg)):
  if(i == len(numList_neg)-1):
    text_1 = text_neg[numList_neg[i]:len(text_neg)]
  else:
    text_1 = text_neg[numList_neg[i]:numList_neg[i+1]]
  text_neg_list.append(text_1)

neg = pd.DataFrame(text_neg_list)
pos

,0
0,0/NR아침/NNG에/JKM일어/NNG나/JX너/NP에게/JKM짧/VA은/ETD인사...
1,1/NR네/MDN가/VV아/ECS맛있/VA었/EPT다고/EFN하/VV었/EPT더/E...
2,2/NR마주치/VV는/ETD눈빛/NNG에/JKM아기/NNG아기/NNG왠지/MAG어색...
3,3/NR어깨동무/NNG까지/JX는/JX괜찮/VA아/ECD치마/NNG손잡/VV는/ET...
4,4/NR햇살/NNG이/JKS가득/MAG그/MDT대/VVㄹ/ETD향/NNG하/XSVㄴ...
...,...
341,735/NR반/NNG쯤/NNG미치/VV어/ECS가/VV아/ECS는/JX다/MAG망가...
342,736/NR무/NNG궁/NNG화/XSN꽃/NNG이/JKS피/VV었/EPT습니다/EF...
343,737/NR아/IC예/IC아예/MAG아예/MAG이렇게/MAG감/NNG을/JKO못/M...
344,738/NR나쁘/VAㄴ/ETD계집애/NNG나/VV아/ECS나/NP는/JX나쁘/VAㄴ...


####okt 이용

In [ ]:
okt = Okt()

In [ ]:
data_pos = Lyrics_pos['Lyrics'].to_string()
data_neg = Lyrics_neg['Lyrics'].to_string()

In [ ]:
pos_pos = okt.pos(data_pos)
list_pos = []
list_neg = []
for i in range(len(pos_pos)):
  pos_text = pos_pos[i][0]+'/'+pos_pos[i][1]
  list_pos.append(pos_text)
  i += 1
  
pos_neg = okt.pos(data_neg)
for i in range(len(pos_neg)):
  neg_text = pos_neg[i][0]+'/'+pos_neg[i][1]
  list_neg.append(neg_text)
  i += 1

KeyboardInterrupt: ignored

In [ ]:
# 형태소분석이 된 긍정가사를 가사별로 리스트화하여 dataframe으로 정리
text = ' '.join(str(i) for i in kkma_pos)
text_pos = text.split('/Number')

pos = pd.DataFrame(text_pos)
pos = pos.drop(0, axis = 0 )
pos.columns = ['word']

# 형태소분석이 된 부정가사를 가사별로 리스트화하여 dataframe으로 정리
text = ' '.join(str(i) for i in kkma_neg)
text_neg = text.split('/Number')

neg = pd.DataFrame(text_neg)
neg = neg.drop(0, axis = 0 )
neg.columns = ['word']

### KoSac 코드 (kkma 이용)

가사별로 형태소분석 list로 나눠주기 => 이후 dataframe화 함

:

In [ ]:
# kosac을 POS 와 NEG로 분류
kosac_pos = kosac[kosac['max.value'] == 'POS']
kosac_neg = kosac[kosac['max.value'] == 'NEG']

In [ ]:
def LyricsWordCount(arrList, emotion):
    list = []
    i=1

    # 감성 점수 구하기
    for Lyrics in arrList:
        neg_count = 0       # 부정 점수
        pos_count = 0       # 긍정 점수
        num = 0             # 감성 점수
        
        # 긍정 점수 구하기
        for pos_text in kosac_pos['ngram']:
            pos_find = Lyrics.find(pos_text)
            if(pos_find > -1):
                pos_count += 1
            else: pos_count += 0


        # 부정 점수 구하기
        for neg_text in kosac_neg['ngram']:
            neg_find = Lyrics.find(neg_text)
            if(neg_find > -1):
                neg_count += 1
            else: pos_count += 0

        # 감성 점수 공식
        if emotion == '긍정':
            num = pos_count - neg_count
        elif emotion == '부정':
            num = neg_count - pos_count

        # 감성 점수 공식
        if emotion == '긍정':
          if pos_count == 0 or neg_count == 0:
            num = 0
          else:
            num = pos_count/(pos_count + neg_count)
            
        elif emotion == '부정':
          if pos_count == 0 or neg_count == 0:
            num = 0
          else:
            num = neg_count/(pos_count + neg_count)

        # 긍, 부정 결과
        if emotion == '긍정':
          if num >= 0.5:
            result = '긍정'
          elif num < 0.5:
            result = '부정'
        
        if emotion == '부정':
          if num >= 0.5:
            result = '부정'
          elif num < 0.5:
            result = '긍정'


        srt = f"{i}번 노래 결과 : 긍정 : {pos_count}, 부정 : {neg_count} , 감성 점수 : {round(num,3)}  {result}"
        i = i + 1

        list.append(srt)
    return list

pos_result = LyricsWordCount(pos['word'], "긍정")
neg_result = LyricsWordCount(neg['word'], "부정")

print('긍정 노래')
for pos in pos_result:
    print(pos)

print('부정 노래')
for neg in neg_result:
    print(neg)

In [ ]:
def Total_pos(list) :
    count = 0
    for i in list:
        if(i[len(i)-2:len(i)] == '긍정'):
            count = count + 1

    return count        

sum = Total_pos(pos_result)
print('긍정 정확도: ', sum/len(Lyrics_pos))

def Total_neg(list) :
    count = 0
    for i in list:
        if(i[len(i)-2:len(i)] == '부정'):
            count = count + 1

    return count        

sum = Total_neg(pos_result)
print('부정 정확도: ', sum/len(Lyrics_pos))

In [ ]:
# 긍정정.csv 파일 저장
import csv
with open('kosac_pos.csv', 'w', newline= '\n') as f:
  writer = csv.writer(f)
  writer.writerow(pos_result)

In [ ]:
# 부정.csv 파일 저장
import csv
with open('kosac_neg.csv', 'w', newline= '\n') as f:
  writer = csv.writer(f)
  writer.writerow(neg_result)